In [1]:
import cv2
import os
import numpy as np
from IPython.display import Image
import imutils

In [2]:
def charger(file):
    img = cv2.imread(file)
    return img

def montrer(img):
    cv2.imshow("image", img)
    k = cv2.waitKey(0)

def inserer(xBegin, yBegin, img, frag, angle=0):
    if angle != 0 and angle != None:
        frag = tourner(frag, angle)
    img[xBegin: xBegin + frag.shape[0], yBegin: yBegin + frag.shape[1]] = frag
    return img

def tourner(img, angle):
    imgRotated = imutils.rotate_bound(img, angle)
    return imgRotated

In [3]:
#récupération du tableau de base
angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
print(angelo.dtype)

uint8


In [4]:
#récupération d'un fragment
fragment = charger("tp1/frag_eroded/frag_eroded_0.png")
print(fragment.shape)

(68, 89, 3)


In [5]:
blank_img = np.zeros(shape=(775, 1707, 3), dtype=np.uint8)
print(blank_img.shape)
blank_img[:,:,:] = 255
montrer(blank_img)

(775, 1707, 3)


In [6]:
imgTestRed = np.zeros(shape=(50, 50, 3), dtype=np.uint8)
imgTestRed[:,:,0] = 255
montrer(imgTestRed)

In [7]:
#test fonction de rotation
montrer(tourner(fragment, 0))

In [8]:
fragment = charger("tp1/frag_eroded/frag_eroded_0.png")
angelo = charger("tp1/Michelangelo_ThecreationofAdam_1707x775.jpg")
montrer(inserer(0, 0, angelo, fragment))
montrer(inserer(0, 0, angelo, fragment, 30))